<a href="https://colab.research.google.com/github/MichalMichniak/pytorch_projects/blob/main/irys_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
print(torch.__version__)
import torch.nn as nn

1.13.1+cu116
